# Models training 
This notebook is used to train test and save models that are defined in the `autoencoders.py` file. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
%load_ext autoreload
%autoreload 2
plt.style.use('ggplot')
import sys 
import os
sys.path.append('/'.join(os.getcwd().split('/')[:4]))
from config.get import cfg
from tensorflow import keras
from tensorflow.keras import layers
import autoencoders

2022-01-11 17:15:12.754487: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-11 17:15:12.754529: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Loading data

In [2]:
X_train = np.load(cfg['files']["liquid"]["scaled_ae_train_features"])
X_test  = np.load(cfg['files']["liquid"]["raw_test_features"])
print(f"shapes : X_train={X_train.shape},X_test={X_test.shape}")

shapes : X_train=(4963, 3, 600, 2),X_test=(1241, 3, 600, 2)


# Loading model
Loading a model defined in `autoencoders.py`

In [27]:
int(3/2)

1

In [45]:
params = {
    "activation" : "selu",
    "dense_layers":3,
    "dropout":0,
    "epochs":70,
    "first_neuron":900,
    "optimizer":"Adamax"
}


def FC():
    model_name = "fully_connected"
    in_shape = (3,600, 2) 
    # build encoder
    input_layer = keras.Input(shape=in_shape)
    x = layers.Reshape([in_shape[0]*in_shape[1]*in_shape[2]])(input_layer)
    x = layers.Dense(in_shape[0]*in_shape[1]*in_shape[2],  activation='selu')(x)
    x = layers.Dense(100,  activation='selu')(x)
    x = layers.Dense(in_shape[0]*in_shape[1]*in_shape[2],  activation='selu')(x)
    output_layer =layers.Reshape(in_shape)(x)
    
    
    # combine encoder and decoder
    autoencoder = keras.Model(input_layer, output_layer)
    autoencoder.compile(optimizer='Adamax', loss='mean_squared_error',)
    return model_name ,autoencoder

model_name,autoencoder = FC()  

#model_name,autoencoder = fully_connected_3L()
autoencoder.summary()
train_loss = []
test_loss = []

Model: "model_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, 3, 600, 2)]       0         
                                                                 
 reshape_40 (Reshape)        (None, 3600)              0         
                                                                 
 dense_62 (Dense)            (None, 3600)              12963600  
                                                                 
 dense_63 (Dense)            (None, 100)               360100    
                                                                 
 dense_64 (Dense)            (None, 3600)              363600    
                                                                 
 reshape_41 (Reshape)        (None, 3, 600, 2)         0         
                                                                 
Total params: 13,687,300
Trainable params: 13,687,300
Non-

# Training the model

In [ ]:
hist = autoencoder.fit(X_train, X_train,epochs=20,validation_data=(X_test, X_test),batch_size=16)

# save losses
train_loss += hist.history["loss"]
test_loss  += hist.history["val_loss"]

Epoch 1/20
311/311 [==============================] - 97s 310ms/step - loss: 0.5321 - val_loss: 0.3777
Epoch 2/20
311/311 [==============================] - 95s 306ms/step - loss: 0.2602 - val_loss: 0.2893
Epoch 3/20
311/311 [==============================] - 90s 291ms/step - loss: 0.2066 - val_loss: 0.2569
Epoch 4/20
311/311 [==============================] - 89s 287ms/step - loss: 0.1838 - val_loss: 0.2389
Epoch 5/20
311/311 [==============================] - 92s 296ms/step - loss: 0.1671 - val_loss: 0.2235
Epoch 6/20
311/311 [==============================] - 94s 301ms/step - loss: 0.1549 - val_loss: 0.2145
Epoch 7/20
311/311 [==============================] - 94s 301ms/step - loss: 0.1460 - val_loss: 0.2107
Epoch 8/20
311/311 [==============================] - 93s 298ms/step - loss: 0.1402 - val_loss: 0.2025
Epoch 9/20
311/311 [==============================] - 93s 298ms/step - loss: 0.1349 - val_loss: 0.2009
Epoch 10/20
311/311 [==============================] - 92s 297ms/step - l

In [ ]:
plt.plot(train_loss,label="train")
plt.plot(test_loss,label="test")
plt.xlabel("epoch")
plt.ylabel("MSE")
plt.legend()
plt.show()

# Saving 
We save the trained model and the recorded losses

In [39]:
autoencoder.save(cfg["models"]["autoencoder"] + f"{model_name}")
np.save(file = cfg["models"]["autoencoder"]+ f"{model_name}_train_loss", arr = np.array(train_loss))
np.save(file = cfg["models"]["autoencoder"]+ f"{model_name}_test_loss", arr = np.array(test_loss))